In [1]:
import sys
sys.path.append("D:\python\lib\site-packages")
import os
import logging
import pyxdf as xdf
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
xdf.load_xdf("/Users/annasharkova/Documents/CSIM/Real-time_interaction/Project/untitled.xdf")

INFO:pyxdf.pyxdf:Importing XDF file /Users/annasharkova/Documents/CSIM/Real-time_interaction/Project/untitled.xdf...
DEBUG:pyxdf.pyxdf: Read tag: 1 at 8 bytes, length=58
DEBUG:pyxdf.pyxdf: Read tag: 2 at 71 bytes, length=1920, StreamId=1
DEBUG:pyxdf.pyxdf:  found stream openvibeSignal
DEBUG:pyxdf.pyxdf: Read tag: 3 at 1996 bytes, length=3892, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,68]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 5893 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 9554 bytes, length=692, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,12]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 10251 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 13912 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 17573 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 21234 bytes, length=692, StreamId=1
DEBUG:

DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 249345 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 253006 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 256667 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 4 at 260325 bytes, length=22, StreamId=1
DEBUG:pyxdf.pyxdf: Read tag: 3 at 260352 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 264013 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 267674 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 271335 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 274996 bytes, length=3884, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,68]
DEBUG:pyxdf.pyxdf: Read tag: 3

([{'info': defaultdict(list,
               {'name': ['openvibeSignal'],
                'type': ['signal'],
                'channel_count': ['14'],
                'nominal_srate': ['128'],
                'channel_format': ['float32'],
                'source_id': ['(0x66fa1316, 0x49bb6f11)'],
                'version': ['1'],
                'created_at': ['46738.0279973'],
                'uid': ['495fbcdc-25ea-4e40-a81a-39e6f9617753'],
                'session_id': ['default'],
                'hostname': ['LAPTOP-JQKQ10DS'],
                'v4address': [None],
                'v4data_port': ['16572'],
                'v4service_port': ['16572'],
                'v6address': [None],
                'v6data_port': ['16572'],
                'v6service_port': ['16572'],
                'desc': [defaultdict(list,
                             {'channels': [defaultdict(list,
                                           {'channel': [defaultdict(list,
                                    

In [100]:
for i in range(len(streams[0])):
    print(streams[0][i]['info']['name'][0])

openvibeSignal


In [5]:
#kind of original
def exportStream(file_path, stream_name, markers_stream_name=None, subject_number=None, markers_to_write=None):
    """
    file_path -- path to the XDF file
    stream_name -- name of the stream (in the XDF file) that is going to be exported
    markers_stream_none -- name of the stream of markers, if any (optional)
    subject_number -- number of the subject, used only name the generated CSV file
    markers_to_write -- list of markers to be exported if only some are desired (if empty, all markers in markers_stream name will be exported)
    """
    streams = xdf.load_xdf(file_path)

    # Find the desired stream
    desired_stream = None
    for i in range (len(streams[0])):
        if streams[0][i]['info']['name'][0] == stream_name:
            desired_stream = streams[0][i]
            break
    
    if desired_stream is None:
        return "Stream " + stream_name + " not found"

    if markers_stream_name is not None and markers_stream_name is not '':
        # Put the data in array, row by row, with a place for the markers
        desired_stream_data = []
        for i in range(len(desired_stream['time_series'])):
            desired_stream_data.append([desired_stream['time_series'][i][0], 0, desired_stream['time_stamps'][i]])
        
        # Find the Marker data
        markers = None
        for i in range (len(streams[0])):
            if streams[0][i]['info']['name'][0] == markers_stream_name:
                markers = streams[0][i]
                break
        
        if markers is None:
            return "Markers stream " + markers_stream_name + " not found"

        # Define function for finding the nearest value in an array
        # Used to find the closest desired stream timestamp that matches the timestamp of each marker
        def find_nearest(array,value):
            idx = (np.abs(array-value)).argmin()
            return idx

        # Add the markers to right rows of desired stream data
        for i in range(len(markers['time_series'])):
            if markers_to_write == None or len(markers_to_write) == 0 or int(markers['time_series'][i]) in markers_to_write:
                index = find_nearest(desired_stream['time_stamps'], markers['time_stamps'][i])
                desired_stream_data[index][1] = int(markers['time_series'][i])

        # Create a dataframe with the desired stream data
        desired_stream_dataframe = pd.DataFrame(desired_stream_data, columns=[stream_name, 'Markers', 'Time'])
    else:
        # Put the data in array, row by row
        desired_stream_data = []
        for i in range(len(desired_stream['time_series'])):
            desired_stream_data.append([desired_stream['time_series'][i][0], desired_stream['time_stamps'][i]])
        
        # Create a dataframe with the desired stream data
        desired_stream_dataframe = pd.DataFrame(desired_stream_data, columns=[stream_name, 'Time'])

    # Save the desired stream dataframe to CSV
    #output_file_path = file_path[:file_path.rfind('/')] # File path without file name
    #output_file_path = os.path.abspath("D:\OpenVive")
    #output_file_name = stream_name
    #if subject_number is not None and subject_number is not '':
        #output_file_name = output_file_name + '-' + str(subject_number)
    #desired_stream_dataframe.to_csv(output_file_path + output_file_name + '.csv', sep='\t', index=False)
    desired_stream_dataframe.to_csv("D:\OpenVive\\untitled-test.csv", sep=';', index=False)
    print("File created successfully")

In [12]:
# We use
def exportStream(file_path, stream_name, markers_stream_name=None, subject_number=None, markers_to_write=None):
    """
    file_path -- path to the XDF file
    stream_name -- name of the stream (in the XDF file) that is going to be exported
    markers_stream_none -- name of the stream of markers, if any (optional)
    subject_number -- number of the subject, used only name the generated CSV file
    markers_to_write -- list of markers to be exported if only some are desired (if empty, all markers in markers_stream name will be exported)
    """
    streams = xdf.load_xdf(file_path)

    # Find the desired stream
    desired_stream = None
    for i in range (len(streams[0])):
        if streams[0][i]['info']['name'][0] == stream_name:
            desired_stream = streams[0][i]
            break
    
    if desired_stream is None:
        return "Stream " + stream_name + " not found"

    if markers_stream_name is not None and markers_stream_name is not '':
        # Put the data in array, row by row, with a place for the markers
        desired_stream_data = []
        for i in range(len(desired_stream['time_series'])):
            desired_stream_data.append([desired_stream['time_series'][i][0], 0, desired_stream['time_stamps'][i]])
        
        # Find the Marker data
        markers = None
        for i in range (len(streams[0])):
            if streams[0][i]['info']['name'][0] == markers_stream_name:
                markers = streams[0][i]
                break
        
        if markers is None:
            return "Markers stream " + markers_stream_name + " not found"

        # Define function for finding the nearest value in an array
        # Used to find the closest desired stream timestamp that matches the timestamp of each marker
        def find_nearest(array,value):
            idx = (np.abs(array-value)).argmin()
            return idx

        # Add the markers to right rows of desired stream data
        for i in range(len(markers['time_series'])):
            if markers_to_write == None or len(markers_to_write) == 0 or int(markers['time_series'][i]) in markers_to_write:
                index = find_nearest(desired_stream['time_stamps'], markers['time_stamps'][i])
                desired_stream_data[index][1] = int(markers['time_series'][i])

        # Create a dataframe with the desired stream data
        desired_stream_dataframe = pd.DataFrame(desired_stream_data, columns=["0", "1", "2", "3"])
    else:
        # Put the data in array, row by row
        desired_stream_data = []
        #for i in range(len(desired_stream['time_series'])):
            #desired_stream_data.append([desired_stream['time_series'][i][0], desired_stream['time_stamps'][i]])
            
        for i in range(len(desired_stream['time_series'])):
            desired_stream_data.append([desired_stream['time_stamps'][i], desired_stream['time_series'][i][0], desired_stream['time_series'][i][1], desired_stream['time_series'][i][2]])
        
        # Create a dataframe with the desired stream data
        desired_stream_dataframe = pd.DataFrame(desired_stream_data, columns=["0", "1", "2", "3"])

    # Save the desired stream dataframe to CSV
    desired_stream_dataframe.to_csv("/Users/annasharkova/Documents/CSIM/Real-time_interaction/Project/untitled-test.csv", sep=';', index=False)
    print("File created successfully")

In [13]:
exportStream("/Users/annasharkova/Documents/CSIM/Real-time_interaction/Project/untitled.xdf", stream['info']['name'][0], "")

INFO:pyxdf.pyxdf:Importing XDF file /Users/annasharkova/Documents/CSIM/Real-time_interaction/Project/untitled.xdf...
DEBUG:pyxdf.pyxdf: Read tag: 1 at 8 bytes, length=58
DEBUG:pyxdf.pyxdf: Read tag: 2 at 71 bytes, length=1920, StreamId=1
DEBUG:pyxdf.pyxdf:  found stream openvibeSignal
DEBUG:pyxdf.pyxdf: Read tag: 3 at 1996 bytes, length=3892, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,68]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 5893 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 9554 bytes, length=692, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,12]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 10251 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 13912 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 17573 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 21234 bytes, length=692, StreamId=1
DEBUG:

DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 249345 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 253006 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 256667 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 4 at 260325 bytes, length=22, StreamId=1
DEBUG:pyxdf.pyxdf: Read tag: 3 at 260352 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 264013 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 267674 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 271335 bytes, length=3656, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,64]
DEBUG:pyxdf.pyxdf: Read tag: 3 at 274996 bytes, length=3884, StreamId=1
DEBUG:pyxdf.pyxdf:  reading [14,68]
DEBUG:pyxdf.pyxdf: Read tag: 3

File created successfully


In [ ]:
datacsv = pd.read_csv('/Users/annasharkova/Documents/CSIM/Real-time_interaction/Project/untitled-test.csv', sep=';')
plt.plot(datacsv['Ch1'])
#print(datacsv['Time'])
#datacsv